In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression


import warnings
warnings.filterwarnings('ignore')


In [2]:
mercado_df = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Mercado/data_consolidado_final_jun24.csv",
                     encoding='latin-1',
                     delimiter=';')

In [3]:
df_mod=mercado_df[["premio_total_liq_cap","ano_mod_conv","classe_bonus",
                   "idade","is_casco","is_dmoral","is_rcdc","is_rcdm",
                   "RNS_Casco_calc",
                   "Grupo_veiculo_new", "fabrica","idade_ult_RNS","md_categoria",
                   "MD_comb","md_config","MD_franquia","md_menor26","md_sexo",
                   "MD_Subzona","md_tipo_renov","MD_uso_veic","uf"]]

df_mod.rename(columns={'premio_total_liq_cap':'premio_ofertado', 'ano_mod_conv': 'ano_mod'},inplace=True)

In [4]:
x_df_num = df_mod[['ano_mod', 'classe_bonus', 'idade', 'is_casco', 'is_dmoral', 'is_rcdc', 'is_rcdm', 'RNS_Casco_calc']]
x_df_cat = df_mod[['Grupo_veiculo_new', 'fabrica', 'idade_ult_RNS', 'md_categoria', 'MD_comb', 'md_config', 'MD_franquia', 'md_menor26', 'md_sexo', 'MD_Subzona','md_tipo_renov', 'MD_uso_veic','uf']]
y   = df_mod[['premio_ofertado']]
x_df_dummy=pd.get_dummies(x_df_cat)
x_df=pd.concat([x_df_num, x_df_dummy],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.15) 

In [5]:
import lightgbm as lgb
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learnnig_rage': 0.05,
    'metric': {'l2','l1'},
    'verbose': -1
}

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval  = lgb.Dataset(x_test, y_test, reference=lgb_train)

model_gbm = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval)

y_pred = model_gbm.predict(x_test, predict_disable_shape_check=True)
print("Premio Estimado Médio: %.2f" % np.mean(y_pred) )
# accuracy check
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(0.5)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % rmse) 

Premio Estimado Médio: 2651.57
MSE: 596991.67
RMSE: 772.65


## modelagem GLM

In [6]:
df_mod_mp_feb23may24_v3 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23may24_v3.csv",
                     encoding='latin-1',
                     delimiter='|')

parte para incluir o preço do mercado na base de modelagem glm

In [7]:
colunas=['premio_ofertado','ano_mod','classe_bonus','MD_idade','is_casco_ofer','is_dmoral','is_rcdc','is_rcdm','RNS_Casco_calc',
 'Grupo_veiculo_new','FABRICA','idade_ult_RNS', 'md_categoria','MD_comb','MD_config','MD_franquia','MD_menor26','MD_sexo','MD_Subzona',  
 'MD_tipo_renov','MD_uso_veic','UF']

In [8]:
df_escorar_gbm=df_mod_mp_feb23may24_v3[colunas]

In [9]:
df_escorar_gbm.rename(columns={'MD_idade':'idade', 
                                   'is_casco_ofer': 'is_casco',
                                   'FABRICA': 'fabrica',
                                   'MD_config':  'md_config',
                                   'MD_menor26': 'md_menor26',
                                   'MD_sexo': 'md_sexo',
                                   'UF': 'uf',
                                   'MD_tipo_renov':  'md_tipo_renov'},inplace=True)
df_escorar_gbm['idade'] = pd.to_numeric(df_escorar_gbm['idade'], errors='coerce')

In [10]:
df_num = df_escorar_gbm[['ano_mod', 'classe_bonus', 'idade', 'is_casco', 'is_dmoral', 'is_rcdc', 'is_rcdm', 'RNS_Casco_calc']]
df_cat = df_escorar_gbm[['Grupo_veiculo_new', 'fabrica', 'idade_ult_RNS', 'md_categoria', 'MD_comb', 'md_config', 'MD_franquia', 'md_menor26', 'md_sexo', 'MD_Subzona','md_tipo_renov', 'MD_uso_veic','uf']]
df_cat_glm=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_num, df_cat_glm],axis=1)
GBM_mercado = model_gbm.predict(df_cat_glm, predict_disable_shape_check=True)

In [11]:
import numpy as np
np.average(GBM_mercado)

4076.1916053985497

In [12]:
df_mod_mp_feb23may24_v3['GBM_mercado']=GBM_mercado

#### prepara base de modelagem glm

In [26]:
df_mod_filtered=df_mod_mp_feb23may24_v3[["premio_ofertado","premio_total",'GBM_mercado', 'premio_creserva','premio_demais_cob',"teste_validacao","ind_renovacao","classe_bonus","MD_sexo","cod_corr",
                                "Cod_End","MD_sin_5a_cas","MD_tempo_casa_2","MD_idade_veic","MD_faixa_IScas",
                                "anomes_dt_ter" ,"MD_idade" ,"per_com", "RNS_Casco_calc","MD_menor26","UF","Renovacao_ant",
                                "fx_ir_12m","is_casco_ofer","MD_fator_ajuste","MD_capac","md_categoria",
                                "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA','MD_est_civil',
                                'peso','Grupo_veiculo_new','vigencia',
                                'Risco_PP_mod', 'Risco_PT_mod','Risco_RF_mod','Risco_DM_mod','Risco_AS_mod',
                                'Risco_DC_mod' ,'Risco_DMO_mod']]

In [27]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['peso'].notna()]
df_mod_filtered = df_mod_filtered.query('premio_total !=0')
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_filtered = df_mod_filtered.query('is_casco_ofer > 0')
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']>0) & (df_mod_filtered['per_com']<50)]
df_mod_filtered = df_mod_filtered.query('premio_total >100')
df_mod_filtered = df_mod_filtered.query('MD_idade !="ERRO"')
df_mod_filtered = df_mod_filtered[(df_mod_filtered.UF != "SP") | (~df_mod_filtered.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total <=2.3) &  (df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total>=0.7)]

df_mod_filtered.shape

(358297, 44)

In [28]:
df_mod_filtered['DA_emissao'] =0.2349
df_mod_filtered['DA_producao'] =0.2749
df_mod_filtered['risco_dmais_cob'] =df_mod_filtered['premio_demais_cob']*0.2627*1.4
df_mod_filtered['premio_anterior'] =df_mod_filtered['premio_total']
df_mod_filtered['Act_cost']  = df_mod_filtered['Risco_PP_mod'] + df_mod_filtered['Risco_PT_mod'] + df_mod_filtered['Risco_RF_mod']+ df_mod_filtered['Risco_DM_mod']+ df_mod_filtered['Risco_AS_mod'] + df_mod_filtered['Risco_DC_mod'] +df_mod_filtered['Risco_DMO_mod'] + df_mod_filtered['risco_dmais_cob']
df_mod_filtered['Premio_IC100']  = df_mod_filtered['Act_cost']/(1-df_mod_filtered['per_com']/100-df_mod_filtered['DA_producao'] )
df_mod_filtered["premio"] = df_mod_filtered["premio_ofertado"]

In [29]:
np.mean(df_mod_filtered['Premio_IC100'])

3028.7706417893046

In [30]:
df_mod=df_mod_filtered.copy()

In [31]:
#print(df_mod['classe_bonus'].value_counts(ascending=False))

10    106996
1      52276
2      37281
3      28712
9      22885
4      22795
5      21800
6      20883
7      19518
8      18450
0       6701
Name: classe_bonus, dtype: int64


In [32]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(int)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01 - 00-<=020' if x == '01 - 00-<=020' else
                                                              '02 - 20-<=040' if x == '02 - 20-<=040' else
                                                              '03 - 40-<=060' if x == '03 - 40-<=060' else
                                                              '04 - 60-<=080' if x == '04 - 60-<=080' else
                                                              '05 - 80-<=100' if x == '05 - 80-<=100' else
                                                              '02 - 20-<=040' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor Mais' if x == 'Corretor Mais'  else 'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: '<10' if x < 10 else
                                                          '>=10' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x: 'Sem Sinistro' if x == 'Sem Sinistro' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else
                                                                      '01-05 ano' if x == '01 ano' else '06+ ano +')


    df['UF_bin'] = df['UF'].apply(lambda x: 'AL,CE,RS'  if x == 'AL' else
                                                'AL,CE,RS'  if x == 'CE' else
                                                'AL,CE,RS'  if x == 'RS' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'AP' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'DF' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'ES' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'PA' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'PB' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'RN' else 
                                                'AP,DF,ES,PA,PB,RN,SC' if x == 'SC' else 
                                                'GO,MA,MG'             if x == 'GO' else 
                                                'GO,MA,MG'             if x == 'MA' else 
                                                'GO,MA,MG'             if x == 'MG' else 
                                                'MT,PR'                if x == 'MT' else
                                                'MT,PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1 - SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2 - RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2 - RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3 - RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3 - RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3 - RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3 - RenovCongenere'   if x == '7 - Renov BB com sinistro' else  '2 - RenovPropria')
       
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: '<101'    if x < 101 else
                                                                    '>=101'   if x >= 101  else '<101')   

    df['MD_qtd_assis_cpf_bin'] = df['MD_qtd_assis_cpf'].apply(lambda x: '00'  if x == 0 else
                                                                        '01'  if x == 1 else
                                                                        '>=2' if x >= 2 else 'ERROR')
     
    return df

In [33]:
df_mod = categorizar(df_mod)

In [34]:
df_train = df_mod.query('teste_validacao == "Trein"')
df_train.shape

(286808, 65)

In [20]:
#df_train2 = df_train[df_train[(df_train['UF'] == 'SP')] & (~df_train['Grupo_veiculo_new'].isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
#df_train2.shape

In [21]:
#df_train = df_train[(df_train['UF'] == 'SP') & (~df_train['Grupo_veiculo_new'].isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]
#df_train.shape

In [22]:
display(df_train[["Act_cost", "Premio_IC100", "per_com","ind_renovacao",'GBM_mercado']].describe().T)

,count,mean,std,min,25%,50%,75%,max
Act_cost,310913.0,1723.935999,1533.558565,155.790548,880.102121,1287.055629,2035.829042,74027.191185
Premio_IC100,310913.0,3024.359106,2669.004613,312.305413,1555.405520,2256.816147,3576.591487,124394.540725
per_com,310913.0,15.434452,5.672286,1.000000,10.000000,15.000000,20.000000,49.000000
ind_renovacao,310913.0,0.464191,0.498717,0.000000,0.000000,0.000000,1.000000,1.000000
GBM_mercado,310913.0,4057.173765,1077.856824,2271.226495,3326.465229,3821.795612,4503.196223,11331.181481


In [ ]:
#df_train.columns.to_list()

In [35]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
formula = 'ind_renovacao ~ 1 + np.log(premio) + np.divide(premio, is_casco_ofer) + np.log(np.divide(premio, GBM_mercado)) + np.log(np.divide(premio, Premio_IC100)) + (np.divide(premio, premio_anterior)+1) + C(md_tipo_renov_ant_bin,Treatment(reference="2 - RenovPropria")) + C(classe_bonus_bin,Treatment(reference="Demais")) + C(canal_bin,Treatment(reference="Demais")) + C(Cod_End_bin,Treatment(reference="Demais")) + C(fator_ajuste_bin,Treatment(reference="<101")) + per_com + C(per_com_bin,Treatment(reference=">=10")) + np.log(MD_tempo_casa_2+1) + C(idade_ult_RNS_bin,Treatment(reference="Sem Sinistro")) + C(MD_comb_bin,Treatment(reference="Demais")) + C(UF_bin,Treatment(reference="Demais")) + C(fx_ir_12m_bin,Treatment(reference="02 - 20-<=040")) + C(MD_tipo_renov,Treatment(reference="2 - Renov MAPFRE sem sinistro"))' 
model = smf.logit(formula , data=df_train).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.558579
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          ind_renovacao   No. Observations:               286808
Model:                          Logit   Df Residuals:                   286780
Method:                           MLE   Df Model:                           27
Date:                Tue, 06 Aug 2024   Pseudo R-squ.:                  0.1904
Time:                        14:33:28   Log-Likelihood:            -1.6020e+05
converged:                       True   LL-Null:                   -1.9787e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===========================================================================================================================================================================
                                                                                                              coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                   0.6533      0.254      2.575      0.010       0.156       1.151
C(md_tipo_renov_ant_bin, Treatment(reference="2 - RenovPropria"))[T.1 - SeguroNovo]                        -0.4921      0.018    -28.071      0.000      -0.526      -0.458
C(md_tipo_renov_ant_bin, Treatment(reference="2 - RenovPropria"))[T.3 - RenovCongenere]                    -0.1632      0.014    -11.914      0.000      -0.190      -0.136
C(classe_bonus_bin, Treatment(reference="Demais"))[T.00]                                                   -0.3207      0.046     -7.028      0.000      -0.410      -0.231
C(canal_bin, Treatment(reference="Demais"))[T.Corretor Mais]                                                0.0415      0.011      3.863      0.000       0.020       0.062
C(Cod_End_bin, Treatment(reference="Demais"))[T.13509]                                                     -0.2640      0.015    -17.721      0.000      -0.293      -0.235
C(fator_ajuste_bin, Treatment(reference="<101"))[T.>=101]                                                   0.1442      0.030      4.839      0.000       0.086       0.203
C(per_com_bin, Treatment(reference=">=10"))[T.<10]                                                          0.1848      0.026      7.237      0.000       0.135       0.235
C(idade_ult_RNS_bin, Treatment(reference="Sem Sinistro"))[T.01-05 ano]                                     -0.4683      0.025    -18.435      0.000      -0.518      -0.419
C(idade_ult_RNS_bin, Treatment(reference="Sem Sinistro"))[T.06+ ano +]                                     -0.4298      0.013    -33.435      0.000      -0.455      -0.405
C(MD_comb_bin, Treatment(reference="Demais"))[T.Gasolina]                                                   0.3706      0.020     18.370      0.000       0.331       0.410
C(UF_bin, Treatment(reference="Demais"))[T.AL,CE,RS]                                                       -0.2355      0.014    -16.604      0.000      -0.263      -0.208
C(UF_bin, Treatment(reference="Demais"))[T.AP,DF,ES,PA,PB,RN,SC]                                           -0.2501      0.015    -17.222      0.000      -0.279      -0.222
C(UF_bin, Treatment(reference="Demais"))[T.GO,MA,MG]                                                       -0.0755      0.016     -4.828      0.000      -0.106      -0.045
C(UF_bin, Treatment(reference="Demais"))[T.MT,PR]                                                          -0.1333      0.015     -8.972      0.000      -0.162      -0.104
C(UF_bin, Treatment(reference="Demais"))[T.RJ]                                                              0.2938      0.021     13.749      0.0

In [183]:
df_train["prob_est"]=model.predict(df_train)

In [191]:
df_train2.head(5)

,premio_ofertado,premio_total,GBM_mercado,premio_creserva,premio_demais_cob,teste_validacao,ind_renovacao,classe_bonus,MD_sexo,cod_corr,...,per_com_bin,idade_ult_RNS_bin,UF_bin,md_tipo_renov_ant_bin,Cod_End_bin,MD_capac_bin,fator_ajuste_bin,MD_qtd_assis_cpf_bin,prob_est,price_elasticity
355,1971.94,2234.090000,2993.661869,134.27,741.79,Trein,1,10,Masculino,91371,...,>=10,01 ano,"MT,PR",2-RenovPropria,Demais,Demais,<101,00,0.567999,248.674714
359,2283.56,1650.310000,3719.703757,55.00,350.24,Trein,1,4,Masculino,51795,...,>=10,Sem Sinistro,Demais,2-RenovPropria,Demais,Demais,<101,01,0.532216,311.824853
360,2598.53,2293.230000,3549.395903,0.00,242.72,Trein,0,3,Masculino,22637,...,>=10,Sem Sinistro,Demais,2-RenovPropria,Demais,Demais,<101,00,0.348641,494.084738
364,1200.65,1195.464754,3001.916824,30.53,499.20,Trein,1,7,Masculino,8950,...,>=10,Sem Sinistro,Demais,2-RenovPropria,Demais,Demais,<101,00,0.758445,84.661308
365,1348.33,1335.650000,3032.554847,62.27,460.21,Trein,1,6,Masculino,8950,...,>=10,Sem Sinistro,Demais,2-RenovPropria,Demais,Demais,<101,>=2,0.764752,92.592248


## Base Para escorar e otimizar

In [24]:
df_lote_092024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Table 7 Base Case Total  - teste novo.csv",
                     encoding='latin-1',
                     delimiter='|')

In [27]:
#df_lote_092024.columns.to_list()

In [31]:
#df_lote_092024.groupby(amostra)['GBM_mercado'].describe()
df_lote_092024.groupby(['Amostra'])['GBM_mercado'].describe()

,count,mean,std,min,25%,50%,75%,max
Amostra,,,,,,,,
Controle,2740.0,2602.944757,1312.284205,1015.431214,1774.339627,2253.495158,2938.428051,10293.704154
Otimização Atu,10925.0,2621.370383,1369.512665,1041.331315,1788.325662,2262.564660,2944.539948,13353.547185
Otimização Mkp,9422.0,2601.304471,1338.159238,1035.647912,1769.360916,2242.979826,2944.864456,12115.226836
Otimização Mkp2,1438.0,3591.388082,1960.647792,1277.207077,2330.196556,3028.914881,3949.047382,11502.727289
Teste_+5%,1409.0,2587.438075,1354.142679,1015.431214,1762.255533,2215.446196,2934.522526,10736.670629
Teste_-5%,1370.0,2631.246128,1359.976155,1064.661439,1820.605352,2286.570944,2962.073010,11740.670563


In [32]:
df_lote_escorar=df_lote_092024.copy()

df_lote_escorar=df_lote_092024[['Premio_oferecido_com_batente','Pr_anterior_TI','Premio_IC100','GBM_mercado',
                                          'Classe_bonus',"MD_sexo","cod_corr","cod_end","MD_tempo_casa_2",
                                          "per_com", "MD_menor26","UF","Renovacao_ant","fx_ir_12m","is_casco_ofer",
                                          "MD_fator_ajuste","md_capac","MD_categoria",
                                          "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA',
                                          'MD_est_civil','Grupo_veiculo_new']]

In [ ]:
#print(df_lote_escorar['UF'].value_counts(ascending=False))

premio|Premio_oferecido_com_batente
is_casco_ofer|is_casco_ofer
GBM_mercado|GBM_mercado
Premio_IC100|Premio_IC100
premio_anterior|Pr_anterior_TI
canal_bin| canal
Cod_End_bin|  cod_end
fator_ajuste_bin|  MD_fator_ajuste
per_com|per_com
MD_tempo_casa_2|MD_tempo_casa_2
idade_ult_RNS_bin|idade_ult_RNS
MD_comb_bin|MD_comb
UF_bin      |     UF
fx_ir_12m_bin|    fx_ir_12m
MD_tipo_renov,|   MD_tipo_renov

In [33]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [36]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [37]:
df_lote_escorar=df_lote_escorar[df_lote_escorar['UF']!= "99"]

In [38]:
df_lote_escorar.shape

(27298, 124)

In [39]:
df_lote = categorizar(df_lote_escorar)

In [40]:
df_lote["prob_est"]=model.predict(df_lote)

In [41]:
df_lote.groupby(["Amostra"])["prob_est"].describe()

,count,mean,std,min,25%,50%,75%,max
Amostra,,,,,,,,
Controle,2740.0,0.471028,0.222370,0.025260,0.293231,0.464536,0.634617,0.971644
Otimização Atu,10921.0,0.464518,0.220433,0.025311,0.286944,0.460232,0.622383,0.971052
Otimização Mkp,9420.0,0.466153,0.222900,0.027128,0.285933,0.459194,0.626422,0.969145
Otimização Mkp2,1438.0,0.493011,0.244844,0.017459,0.283457,0.504693,0.688627,0.975652
Teste_+5%,1409.0,0.444944,0.225539,0.041454,0.258744,0.431881,0.605808,0.947400
Teste_-5%,1370.0,0.497834,0.224050,0.018847,0.324497,0.495979,0.663623,0.991697


In [42]:
df_lote.groupby(["Amostra"])["Renewal Demand"].describe()

,count,mean,std,min,25%,50%,75%,max
Amostra,,,,,,,,
Controle,2740.0,0.514640,0.238073,0.016523,0.325433,0.526346,0.706608,0.991850
Otimização Atu,10921.0,0.506352,0.236870,0.022698,0.311941,0.516110,0.690852,0.987610
Otimização Mkp,9420.0,0.507511,0.239024,0.020030,0.312794,0.515957,0.697373,0.980289
Otimização Mkp2,1438.0,0.550326,0.269660,0.011015,0.315279,0.606889,0.770266,0.986715
Teste_+5%,1409.0,0.481886,0.240136,0.035542,0.278098,0.483086,0.674171,0.971993
Teste_-5%,1370.0,0.543340,0.240046,0.020685,0.360677,0.564239,0.741068,0.995868


## Otimização Teste 

In [43]:
print(df_lote['Amostra'].value_counts(ascending=False))

Otimização Atu     10921
Otimização Mkp      9420
Controle            2740
Otimização Mkp2     1438
Teste_+5%           1409
Teste_-5%           1370
Name: Amostra, dtype: int64


In [44]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra == "Otimização Atu"')
df_lote_otim_atu.shape

(10921, 137)

In [45]:
df_lote_otim = df_lote_otim_atu

In [152]:
df_lote_otim = df_lote_otim_atu.sample(n = 1000) 

In [153]:
#df_lote_otim = df_otim1
df_lote_otim.shape

(1000, 141)

In [154]:
df_lote_otim["Margem Inicial"] = df_lote_otim["prob_est"]*(df_lote_otim["Premio_oferecido_com_batente"]-df_lote_otim["Act_cost"])

In [155]:
print('Premio Inicial: %2f' % np.mean(df_lote_otim["premio"]))
print('Probabilidade Inicial: %2f' % np.mean(df_lote_otim["prob_est"]))
print('Margem Inicial: %2f' % np.mean(df_lote_otim["Margem Inicial"]))

Premio Inicial: 2834.260454
Probabilidade Inicial: 0.467046
Margem Inicial: 449.300836


In [166]:
%%time

def margem_esperada(premio,demanda_alvo):
    df_lote_otim["premio"]=premio
    demanda= model.predict(df_lote_otim)
    df_lote_otim["demanda_alvo"]=demanda_alvo
    custo= df_lote_otim["Act_cost"]
    margem_esperada = demanda * (premio-custo)
    margem_total = -np.sum(margem_esperada) 
    return margem_total

# restricão demanda alvo
def constraint(premio,demanda_alvo):
    df_lote_otim["premio"]=premio
    demanda= model.predict(df_lote_otim)
    demanda_alvo = df_lote_otim["demanda_alvo"]
    return  np.sum(demanda) - np.sum(demanda_alvo)


def constraint2(premio,demanda_alvo):
    margem_inicial = df_lote_otim["prob_est"]*(df_lote_otim["Premio_oferecido_com_batente"]-df_lote_otim["Act_cost"])
    df_lote_otim["premio"]=premio
    df_lote_otim["demanda"] = model.predict(df_lote_otim)
    demanda_alvo = df_lote_otim["demanda_alvo"]
    margem_otim = df_lote_otim["demanda"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])
    return  np.sum(margem_otim) - np.sum(margem_inicial)


def constraint3(premio,demanda_alvo):
    lb = df_lote_otim["Premio_oferecido_com_batente"]*0.90
    return lb

def constraint4(premio,demanda_alvo):
    ub = df_lote_otim["Premio_oferecido_com_batente"]*1.1
    return ub
 
restr_demanda = {'type': 'eq', 'fun': constraint}   
restr_margem  = {'type': 'ineq', 'fun': constraint2} 
restr_lb = {'type': 'ineq', 'fun': constraint3}   
restr_ub  = {'type': 'ineq', 'fun': constraint4} 
 
#restricoes = ([restr_demanda,restr_margem,restr_lb,restr_ub])     

restricoes = ([restr_demanda,restr_margem])    

demanda_base_case = np.round(np.mean(df_lote["prob_est"]),2)

demanda_alvo=0.40




#premio_inicial = df_lote_otim["Premio_oferecido_com_batente"]*0.939026
premio_inicial = df_lote_otim["Premio_oferecido_com_batente"]*(demanda_base_case/demanda_alvo)
# limitando o premio fique dentro do range
bounds = [(0.9 * premio_base, premio_base*1.1) for premio_base in df_lote_otim["Premio_oferecido_com_batente"]]



# rodando a otimizaçao
result = minimize(margem_esperada,
                  premio_inicial,
                  args=(demanda_alvo), 
                  method = 'L-BFGS-B', 
                  tol=0.01,
                  constraints=restricoes,
                  bounds=bounds)

# premio otimizados
premio_otim = result.x
df_lote_otim["premio"]=result.x

df_lote_otim["demanda otimizada"] = model.predict(df_lote_otim)
df_lote_otim["Margem otimizada"] = df_lote_otim["demanda otimizada"]*(df_lote_otim["premio"]-df_lote_otim["Act_cost"])

CPU times: total: 4min 46s
Wall time: 1min 42s


In [167]:
print(np.mean(df_lote_otim["Premio_oferecido_com_batente"]))
print(np.mean(df_lote_otim["premio"]))

2862.6345015936
3148.8889353756294


In [168]:
print(np.mean(df_lote_otim["Premio_oferecido_com_batente"]))
print(np.mean(df_lote_otim["premio"]))
print(np.mean(df_lote_otim["Margem Inicial"]))
print(np.mean(df_lote_otim["Margem otimizada"]))
print(np.mean(df_lote_otim["prob_est"]))
print(np.mean(df_lote_otim["demanda otimizada"]))


2862.6345015936
3148.8889353756294
449.3008364795254
488.2667757711094
0.4670461795716562
0.4147380094878265


In [171]:
prices_df = pd.DataFrame({
    'chave': df_lote_otim["chave"],
    'Premio ofertado com batente':  df_lote_otim["Premio_oferecido_com_batente"],
    'Optimized Prices': df_lote_otim["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_lote_otim["prob_est"],
    'renewal demand':df_lote_otim["demanda otimizada"],
    'variação':df_lote_otim["premio"]/ df_lote_otim["Premio_oferecido_com_batente"]
})
prices_df.head(20)

,chave,Premio ofertado com batente,Optimized Prices,lb_bound,up_bound,prob_est,renewal demand,variação
19395,335509028013100001,4211.923349,4633.115684,3790.731014,4633.115684,0.492561,0.432129,1.100000
2018,605401733573100001,1237.204132,1360.924545,1113.483719,1360.924545,0.662164,0.605021,1.100000
9576,17024467073100001,1771.729692,1948.902661,1594.556723,1948.902661,0.495377,0.436557,1.100000
24633,677600090033100001,4342.155000,4776.370500,3907.939500,4776.370500,0.203998,0.159331,1.100000
7087,34040544953100001,2046.519416,2251.148074,1841.867474,2251.171358,0.277603,0.225297,1.099989
18653,8800010663100001,3768.056275,4144.838619,3391.250648,4144.861902,0.286582,0.232153,1.099994
5929,335509014333100001,1573.448851,1730.793736,1416.103966,1730.793736,0.570267,0.513965,1.100000
5138,157700256343100001,1569.238018,1726.161820,1412.314216,1726.161820,0.495368,0.435554,1.100000
20831,606100603823100001,4794.967723,5274.464495,4315.470951,5274.464495,0.430463,0.366454,1.100000
22018,23036641193100001,2997.588088,3297.346897,2697.829279,3297.346897,0.559051,0.501907,1.100000


In [213]:
df_plot = df_lote_otim[["prob_est","premio","Amostra"]].copy()
df_plot.dtypes


prob_est    float64
premio      float64
Amostra      object
dtype: object

In [220]:
df_plot.groupby(['Amostra'])['prob_est'].mean()

Amostra
Otimização Atu    0.467046
Name: prob_est, dtype: float64

In [222]:
# teste
teste = df_plot.groupby(['Amostra'], as_index=False).agg({'prob_est':['mean','std']})
teste


Amostra  prob_est          
                       mean       std
0  Otimização Atu  0.467046  0.218938

In [234]:
#df_plot["range_preco"] = list(range(np.min(df_plot["premio"].astype('int')), np.max(df_plot["premio"].astype('int')), 500))



In [235]:
#plt.figure(figsize=(9,6))
#plt.plot(preco, np.mean(prices_df['renewal demand']), color='grey')
#plt.xlabel('Preço')
#plt.ylabel('Renovação');
#plt.title('Preço Renov');

In [236]:
prices_df["variação"].describe()

count    1000.000000
mean        1.099997
std         0.000006
min         1.099949
25%         1.100000
50%         1.100000
75%         1.100000
max         1.100000
Name: variação, dtype: float64

#criando a função

In [160]:
df_relatorio = df_lote_otim.copy()
df_relatorio

,Act_cost,Amostra,Amostra_inicial,ano_fab,ano_modelo,batente_max_ti,batente_min_ti,batente_min_ti_original,canal,Carprice,...,md_tipo_renov_ant_bin,Cod_End_bin,MD_capac_bin,fator_ajuste_bin,MD_qtd_assis_cpf_bin,prob_est,Margem Inicial,demanda_alvo,demanda otimizada,Margem otimizada
19395,2632.873286,Otimização Atu,Otimização Atu,2012,2012,2.3,0.5,0.6,Sicredi,123566,...,2-RenovPropria,Demais,Demais,<101,00,0.492561,777.779217,0.4,0.432129,864.363035
2018,711.516097,Otimização Atu,Otimização Atu,2024,2024,2.3,0.5,0.6,CorretorMais,84744,...,2-RenovPropria,13509,Demais,<101,00,0.662164,348.091933,0.4,0.605021,392.905954
9576,940.965639,Otimização Atu,Otimização Atu,2022,2023,2.3,0.5,0.6,Corretor,124965,...,2-RenovPropria,Demais,Demais,<101,00,0.495377,411.541297,0.4,0.436557,440.022148
24633,3322.218677,Otimização Atu,Otimização Atu,2021,2021,2.3,0.5,0.6,GoiasForte,120369,...,2-RenovPropria,Demais,Demais,<101,00,0.203998,208.065002,0.4,0.159331,231.691313
7087,1166.720719,Otimização Atu,Otimização Atu,2013,2013,2.3,0.5,0.6,Corretor,33263,...,2-RenovPropria,Demais,Demais,<101,00,0.277603,244.234441,0.4,0.225297,244.318092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,597.357465,Otimização Atu,Otimização Atu,2013,2014,2.3,0.5,0.6,CorretorMais,46125,...,2-RenovPropria,Demais,Demais,<101,00,0.635817,227.788681,0.4,0.580750,263.557860
12153,1421.916781,Otimização Atu,Otimização Atu,2021,2022,2.3,0.5,0.6,C6SEGAssessorias,71673,...,2-RenovPropria,Demais,Demais,<101,00,0.146306,141.544417,0.4,0.114475,138.099982
26200,4321.162302,Otimização Atu,Otimização Atu,2022,2022,2.3,0.5,0.6,BRInfinite,253916,...,2-RenovPropria,Demais,Demais,<101,00,0.205309,532.076554,0.4,0.167785,550.817345
17302,3029.115102,Otimização Atu,Otimização Atu,2011,2011,2.3,0.5,0.6,CorretorMais,114298,...,2-RenovPropria,Demais,Demais,<101,00,0.387594,704.139614,0.4,0.323392,744.213188


In [161]:
df_relatorio["per_com_pct"] = df_relatorio['per_com']/100
df_relatorio["premio_prob"] = df_relatorio["demanda otimizada"]*df_relatorio["premio"]
df_relatorio["Act_cost_prob"] = df_relatorio["demanda otimizada"]*df_relatorio["Act_cost"]                                                                
df_relatorio["comissao_valor"] = df_relatorio["per_com_pct"] *df_relatorio["premio"]
df_relatorio["comissao_valor_prob"] = df_relatorio["per_com_pct"] *df_relatorio["premio_prob"]


In [164]:
apolice_ofertada  = np.round(df_relatorio.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio["demanda otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio["premio"])
retencao_estimada = np.mean(df_relatorio["demanda otimizada"])
retencao_base_case = np.mean(df_relatorio["prob_est"])
premio_esperado   = np.sum(df_relatorio["premio_prob"])/np.sum(df_relatorio["demanda otimizada"])
risco_ofertado    = np.mean(df_relatorio["Act_cost"])
risco_esperado    = np.sum(df_relatorio["Act_cost_prob"])/np.sum(df_relatorio["demanda otimizada"])
sp_ofertado       = np.sum(df_relatorio["Act_cost"])/np.sum(df_relatorio["premio"])
sp_esperado       = np.sum(df_relatorio["Act_cost_prob"])/np.sum(df_relatorio["premio_prob"])
comis_ofertado    = np.sum(df_relatorio["comissao_valor"])/np.sum(df_relatorio["premio"])
comis_esperado    = np.sum(df_relatorio["comissao_valor_prob"])/np.sum(df_relatorio["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

In [165]:
print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_baseCase : %s' % np.round(retencao_base_case*100,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

Apolices Ofertadas: 1000
Apolices Renovadas: 415.0
premio_ofertado: 3148.89
retencao_baseCase : 46.7
retencao_estimada : 41.47
premio_esperado" : 2547.72
risco_ofertado" : 1715.12
risco_esperado" : 1370.43
sp_ofertado" : 54.47
sp_esperado" : 53.79
ic_ofertado" : 98.04
ic_esperado" : 97.54
